<a href="https://colab.research.google.com/github/DiaPorntipa/Bushfire_data_analytics/blob/main/Acclimatised_data_compilation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
working_dir = '../..'  # This repository's root directory
input_file_name = 'in-situ_topography_pcs.csv'
daily_min_output_file_name = 'sentinel2_dfmc_pcs_min_dfmc.csv'
nearest_time_output_file_name = 'sentinel2_dfmc_pcs_nearest_time.csv'

sentinel2_path = 'Data/sentinel2/FMC_actparks_forest_corp_sites_DEA_fmc_est.csv'

In [ ]:
import sys

sys.path.append(working_dir)
import os

from tqdm import tqdm

tqdm.pandas()
import pandas as pd

from Utils.daily_minmax import get_daily_extreme_observation
from Utils.datetime import add_UTC_Datetime

# Loading in-situ data


In [ ]:
# Load in-situ_topography.csv as the main df

df = pd.read_csv(os.path.join(working_dir, "output", "csv", input_file_name))
df = df[~df['DFMC'].isna()]
df['Datetime'] = pd.to_datetime(df['Datetime'])
if 'UTC_Datetime' in df.columns:
    df['UTC_Datetime'] = pd.to_datetime(df['UTC_Datetime'])
else:
    df = add_UTC_Datetime(df)
df.head()

# Loading Sentinel-2 data

In [ ]:
sentinel2_data_dir = os.path.join(working_dir, sentinel2_path)
sentinel2_df = pd.read_csv(sentinel2_data_dir)
sentinel2_df.head()

# Combining in-situ and remote data into a single dataframe

In [ ]:
# Selecting only obs with has the daily min dfmc value

variable = 'DFMC'
date_column_name = 'sentinel2_observation_date'
df[date_column_name] = df['Datetime'].dt.date
df_min_dfmc = get_daily_extreme_observation(df, variable, date_column_name, option='min')
df_min_dfmc.head()

In [ ]:
# Select the first observations if there are multiple ones that has daily minimum dfmc

df_min_dfmc = df_min_dfmc.loc[
    df_min_dfmc.groupby(['SiteID', 'sentinel2_observation_date'])['Datetime'].idxmin()
]
df_min_dfmc.head()

In [ ]:
# TODO (HIGH): Make df_nearest_time, consisting of one nearest site obs to one Sentinel-2 obs
df_nearest_time

In [ ]:
# Fill in df_min_dfmc with sentinel2 data


# For each row, open sentinel2 data csv file one-by-one to get data
def get_sentinel2_value(row, sentinel2_df):
    # TODO (HIGH):

    return
    # SiteID_str = str(row['SiteID'])
    # SiteID_str = ''.join(c for c in SiteID_str if c.isdigit())

    # target_time = row['UTC_Datetime'].round('1d').strftime('%Y-%m-%d')
    # sentinel2_value = sentinel2_df.loc[sentinel2_df['Date'] == target_time, SiteID_str].values[0]

    # return sentinel2_value


df_min_dfmc['sentinel2_DFMC'] = df_min_dfmc.progress_apply(
    lambda row: get_sentinel2_value(row, sentinel2_df), axis=1
)
df_min_dfmc.head()

In [ ]:
# Investigate the result - There are only NaNs in veg_cover column
df_min_dfmc[df_min_dfmc['sentinel2_DFMC'].isna()]

In [ ]:
# Investigate the result - There are only NaNs in veg_cover column
df_nearest_time[df_nearest_time['sentinel2_DFMC'].isna()]

# Save the resulting dataframes

In [ ]:
df_min_dfmc.to_csv(
    os.path.join(working_dir, "output", "csv", daily_min_output_file_name), index=False
)
df_nearest_time.to_csv(
    os.path.join(working_dir, "output", "csv", nearest_time_output_file_name), index=False
)